# IPO Random Forest Regression
Predicting Excess Returns using pre-IPO Data and Random Forest Regressions

In [1]:
import psycopg2
import os

conn = psycopg2.connect(os.environ.get('DB_CONNECTION_STRING'))

In [2]:
cur = conn.cursor()
cur.execute('SELECT * FROM companies;')
cur.fetchone()

(549,
 'BLCT',
 'BlueCity Holdings Ltd.',
 'NASDAQ',
 'Packaged Software',
 'https://www.blued.com/en/index.html',
 'BlueCity Holdings Ltd. designs and develops mobile application software solutions. It develops Blued mobile application, which allow users to connect with each other, express themselves, and access professional health related services and family planning services at their fingertips. The company was founded by Bao Li Ma in 2000 and is headquartered in Beijing, China.',
 '',
 'BlueCity Holdings Ltd ADR',
 'ad',
 'Technology Services',
 7372,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 datetime.datetime(2020, 7, 7, 11, 0, 0, 709000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),
 datetime.datetime(2020, 7, 7, 11, 0, 0, 709000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)))

In [4]:
cur.description

(Column(name='id', type_code=23),
 Column(name='symbol', type_code=1043),
 Column(name='companyName', type_code=1043),
 Column(name='exchange', type_code=1043),
 Column(name='industry', type_code=1043),
 Column(name='website', type_code=1043),
 Column(name='description', type_code=25),
 Column(name='CEO', type_code=1043),
 Column(name='securityName', type_code=1043),
 Column(name='issueType', type_code=1043),
 Column(name='sector', type_code=1043),
 Column(name='primarySicCode', type_code=23),
 Column(name='employees', type_code=23),
 Column(name='address', type_code=1043),
 Column(name='address2', type_code=1043),
 Column(name='state', type_code=1043),
 Column(name='city', type_code=1043),
 Column(name='zip', type_code=1043),
 Column(name='country', type_code=1043),
 Column(name='phone', type_code=1043),
 Column(name='createdAt', type_code=1184),
 Column(name='updatedAt', type_code=1184))